# Chapter 10: Splitting the Difference: Differential Power Analysis

This is the companion notebook to Chapter 10 of the Hardware Hacking Handbook by Jasper van Woudenberg and Colin O'Flynn. The headings in this notebook follow the headings in the book.

© 2021. This work is licensed under a [CC BY-SA 4.0 license](https://creativecommons.org/licenses/by-sa/4.0/). 

## A DPA Attack in Python
### Simulating a Single Power Measurement

In [ ]:
import numpy as np
import random
import pylab as py

# Make this lab repeatable
random.seed(0) 
np.random.seed(0)

#Generate a random lookup table as sbox
lookup = [x for x in range(0,256)]
random.shuffle(lookup)

# Create a simulated power trace, based on data in (din)
def measure_power(din):
    #secret byte
    skey = 0b10010111 # 0x97 
    
    #Calculate result
    res = lookup[din ^ skey]
    
    #Generate some arbitrary random data, not necessarily around zero
    b = 148
    a = 154    
    basetrace = (b - a) * np.random.random_sample(50) + a
    
    #Determine number of 1's in result (Hamming Weight), create appropriately sized spike
    time_of_leakage = 35
    basetrace[time_of_leakage] += bin(res).count('1') - 4 # On average, HW=4, so we minus it out
        
    return basetrace

### The Batch Measurement

In [ ]:
# Create a number of simulated traces and input data
def gen_traces(number_traces):    
    traces = np.array([None] * number_traces)
    input_data = np.random.randint(0, 256, number_traces)

    for d in range(0, number_traces):
        traces[d] = measure_power(input_data[d])
    return (input_data, traces)

In [ ]:
(input_data, traces) = gen_traces(1000)

### Various trace pics from the chapter

In [ ]:
# Helper for plotting
def trcplot(trace, title="", xlabel="", ylabel="", label="", axs=None):
    if not axs:
        fig, axs = py.subplots()
    axs.set_title(title)
    axs.set_xlabel(xlabel)
    axs.set_ylabel(ylabel)
    axs.plot(trace, label=label)

In [ ]:
# Plot traces
lines = 6
fig, axs = py.subplots(lines)
fig.suptitle("Traces")
for trace in range(lines):
    # Get data and perform lookup; format it
    data = input_data[trace]
    datastr = hex(data)

    # Plot it
    curaxs = axs[trace]
    trcplot(traces[trace], ylabel=datastr, axs=curaxs)
    curaxs.set_xticks([])
    curaxs.set_yticks([])
py.savefig("fig/10_6.svg", format="svg")

In [ ]:
trcplot(traces[0], "Trace 0. Input="+hex(input_data[0]), "Sample Number", "Power Measurement")
py.savefig("fig/10_9.svg", format="svg")

In [ ]:
# Calculate difference of means on given input/traces for bit bitnum; also return grouping for each guess
def dom(input_data, traces, bitnum):    
    groups = [0] * 256
    diff_vect = [0] * 256
    traces = np.array(traces) # Convert to numpy so we can slice better below

    # Group each trace based on each guess
    for guess in range(0, 256):
        # Vector: for each trace, apply lookup table
        transformed = np.array([lookup[guess ^ p] for p in input_data])
        # Vector: for each trace, 0 or 1
        group = transformed >> bitnum & 1
        # Save grouping
        groups[guess] = group
        
        # Calculate means for group zero and one
        mean_zero = np.mean(traces[group==0],axis=0)
        mean_one  = np.mean(traces[group==1],axis=0)
        
        # Calc difference
        diff_vect[guess] = mean_one - mean_zero
        
    return (diff_vect, groups)

### The Difference Array (and some more pics)

In [ ]:
# Do DPA
(diff_vect,groups) = dom(input_data, traces, 0)

In [ ]:
# Make a nice DoM plot for a particular guess
def plot_dom_guess(input_data, traces, diff_vect, groups, guess, ylim=None):
    lines = 3
    fig, axs = py.subplots(lines+1,2)
    fig.suptitle("Guess " + hex(guess))
    means = [None, None]

    # For each group {0,1}
    for group in range(2):
        # Obtain traces and data for group
        group_traces = traces[groups[guess]==group]
        group_data = input_data[groups[guess]==group]
        
        # Plot traces
        for trace in range(lines):
            # Get data and perform lookup; format it
            data = lookup[guess ^ group_data[trace]] 
            datastr = bin(data)[2:].zfill(8) 
            
            # Plot it
            curaxs = axs[trace,group]
            trcplot(group_traces[trace], ylabel=datastr, axs=curaxs)
            curaxs.set_xticks([])
            curaxs.set_yticks([])

        # Plot mean for group
        axs[0,group].set_title("Group " + str(group))
        curaxs = axs[lines,group]
        curaxs.set_xticks([])
        curaxs.set_yticks([])
        means[group] = np.mean(group_traces,axis=0)
        trcplot(means[group], ylabel="mean " + str(group), axs=curaxs)
    py.savefig("fig/10_8x_{}_top.svg".format(hex(guess)), format="svg")
        
    # Plot difference of means
    trcplot(diff_vect[guess], ylabel="Difference of Means") 
    if ylim:
        py.ylim(ylim)
    py.savefig("fig/10_8x_{}_bottom.svg".format(hex(guess)), format="svg")
    
# Correct guess
plot_dom_guess(input_data, traces, diff_vect, groups, 0x97)
ylim = py.ylim() # Get y scale

# Incorrect guess
plot_dom_guess(input_data, traces, diff_vect, groups, 0xAB, ylim)

In [ ]:
# Now calculate it with lots more traces  
(input_dataB, tracesB) = gen_traces(100000)
(diff_vectB,_) = dom(input_dataB, tracesB, 0)

In [ ]:
# Plot DoM with 1000 traces  
fig,axs=py.subplots(1,2)
trcplot(diff_vect[0x97], "Guess=0x97 (1000 traces)", "Sample Number", "Difference of Means", axs=axs[0]) 

# Now plot it with lots more traces   
trcplot(diff_vectB[0x97], "Guess=0x97 (100000 traces)", "Sample Number", axs=axs[1])
axs[0].set_ylim(axs[1].get_ylim())

py.savefig("fig/10_10.svg", format="svg")

### A Complete Attack

In [ ]:
# Return best guess for key given input data and traces, and specific bit number
def best_guess(input_data, traces, bitnum):
    # Calculate DoM for given bit and input
    (diff_vect,groups) = dom(input_data, traces, bitnum)
    
    # Vector: For each of the 256 guesses the strongest peak in the DoM trace
    guess_max = np.max(np.abs(diff_vect),axis=1)
    
    # Return guess for which the peak is the highest of all guesses
    return np.argmax(guess_max)

In [ ]:
for bitnum in range(8):
    best_key = best_guess(input_data,traces,bitnum)
    print("Best guess from bit %d: 0x%02x"%(bitnum, best_key))

## Attacking AES-128 using DPA

In [ ]:
# The AES sbox
sbox = (
            0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
            0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
            0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
            0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
            0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
            0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
            0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
            0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
            0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
            0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
            0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
            0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
            0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
            0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
            0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
            0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
            )

# We just replace the lookup table
lookup = sbox

In [ ]:
# Create AES traces and re-guess key
(input_data, traces) = gen_traces(1000)
for bitnum in range(8):
    best_key = best_guess(input_data,traces,bitnum)
    print("Best guess from bit %d: 0x%02x"%(bitnum, best_key))

In [ ]:
# Get diff_vect for all guesses
(diff_vect,groups) = dom(input_data, traces, bitnum)

# Plot 3 guesses
fig, axs=py.subplots()
for guess in range(0x96,0x99):
    trcplot(diff_vect[guess], "AES-128 DPA", "Sample Number", "Difference of Means", label=hex(guess), axs=axs)
axs.legend()
py.savefig("fig/10_12.svg", format="svg")

## Correlation Power Analysis (CPA) Attack 
### Calculating the Data to Correlate

In [ ]:
# Generate a handful of traces for CPA
D = 100
(input_data_AES, traces_AES) = gen_traces(D)
T = len(traces_AES[0])

In [ ]:
# Print table t
for d in [0,1,2,D-1]:
    print("0x%02x" % input_data_AES[d], end="\t")
    for j in [0, 1, T-1]:
        print("%.2f" % traces_AES[d][j], end="\t")
    print()

### Bring in the functions

In [ ]:
# Cache hamming weights for each byte value
HW = [bin(n).count("1") for n in range(0,256)]

# Return hamming weight of sbox[pt xor keyguess]
def intermediate(pt, keyguess):
    return HW[sbox[pt ^ keyguess]]

### Calculating the Correlation

In [ ]:
# Print table p
I = 256
for d in [0,1,2,D-1]:
    print("0x%02x" % input_data_AES[d], end="\t")
    for i in [0, 1, 2, I-1]:
        print(intermediate(input_data_AES[d], i), end="\t")
    print()

### Attacking AES-128 using CPA

In [ ]:
# Do CPA attack, returning 
def cpa(input_data, traces):
    cpaoutput = [0]*256
    maxcpa = [0]*256
    number_traces = len(traces)
    numpoint = len(traces[0])

    for guess in range(0, 256):
        #Initialize arrays & variables to zero
        sumnum = np.zeros(numpoint)
        sumden1 = np.zeros(numpoint)
        sumden2 = np.zeros(numpoint)
        hyp = np.zeros(number_traces)

        # Calculate all hypothetical intermediates for this guess
        for tnum in range(0, number_traces):
            hyp[tnum] = intermediate(input_data[tnum], guess)

        # Calculate means for trace and hypothesis
        meanh = np.mean(hyp, dtype=np.float64) # h_i hat
        meant = np.mean(traces)                # t_j hat

        # For each trace, update sums
        for tnum in range(number_traces):
            hdiff = hyp[tnum] - meanh     # h_d,i - h_i hat
            tdiff = traces[tnum] - meant  # t_d,j - t_j hat

            sumnum  = sumnum  + hdiff * tdiff # cross-multiply
            sumden1 = sumden1 + hdiff * hdiff # square hdiff
            sumden2 = sumden2 + tdiff * tdiff # square tdiff

        # Finalize correlation calculation
        cpaoutput[guess] = sumnum / np.sqrt(sumden1 * sumden2)

        # Find highest (absolute) correlation peak in correlation trace for this key guess
        maxcpa[guess] = max(abs(cpaoutput[guess]))
    return (cpaoutput,maxcpa)

In [ ]:
# Do CPA
(cpaoutput,maxcpa) = cpa(input_data_AES, traces_AES)
print("Most likely key = 0x%02x" % np.argmax(maxcpa))

In [ ]:
# Print table r
for i in [0, 1, 0x97, I-1]:
    for j in [0, 35, T-1]:
        print("%.2f" % cpaoutput[i][j], end="\t")
    print()

### Correlation Calculation and analysis

In [ ]:
# Plot correlation traces
fig,axs = py.subplots()
for guess in range(0x96, 0x99):
     trcplot(cpaoutput[guess], "AES-128 CPA", "Sample Number", "Correlation", label=hex(guess), axs=axs)
py.legend()
py.savefig("fig/10_13.svg", format="svg")